In [1]:
import hw1_helpers as hh
import math
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots



from pint import UnitRegistry

## constants

- source: fluid properties https://www.climate-policy-watcher.org/wastewater-sludge/physical-and-biological-properties.html

In [2]:
# example constants 
ex_c = hh.constants(1, 1,  1, 1, 1, override=True)
ex_c

{'rho': 998.0 <Unit('kilogram / meter ** 3')>,
 'mu': 0.00102 <Unit('kilogram / meter / second')>,
 'D': 0.06 <Unit('meter')>,
 'h': 1 <Unit('meter')>,
 'beta': 1 <Unit('meter')>,
 'H': 2.2 <Unit('meter')>,
 'x ': 66.66666666666667 <Unit('meter')>,
 'theta': 89.14062775635531 <Unit('degree')>,
 'L': 65 <Unit('meter')>,
 'g': 9.8 <Unit('meter / second ** 2')>,
 'alpha': 2}

In [9]:
# real constants 
# density of primary sludge => 1.0 -> 1.03 g/cm3 =>=> 1000 kg/m3 -> 1030 kg/m3
# dynamic viscosity => 0.04 - 0.11 Pa * s ~ kg/(m*s)
# worst case material scenario
rc = hh.constants(rho=1030, mu=0.04, D=0.1, h=1.1, beta=1.1 )
rc

{'rho': 1030.0 <Unit('kilogram / meter ** 3')>,
 'mu': 0.04 <Unit('kilogram / meter / second')>,
 'D': 0.1 <Unit('meter')>,
 'h': 1.1 <Unit('meter')>,
 'beta': 1.1 <Unit('meter')>,
 'H': 2.2 <Unit('meter')>,
 'x ': 73.33333333333334 <Unit('meter')>,
 'theta': 89.14062775635531 <Unit('degree')>,
 'L': 73.34158286932303 <Unit('meter')>,
 'g': 9.8 <Unit('meter / second ** 2')>,
 'alpha': 2}

# calculations

In [26]:
def calc_properties(d): 
    """calculates properties of the system given dictionary d of appropriate constants """
    
    print(f"pipe height = {d['h']}, diam = {d['D']}, length = {d['L']} ")

    # coefficients
    a = 1
    b = (64 * d["mu"] * d["L"] ) / (d["rho"] * d["D"]**2 * d["alpha"] )
    c =  -(2 * d["g"] * d["H"]) / (d["alpha"])

    # coeff = {
    #     "a": a,
    #     "b": b,
    #     "c": c
    # }

    # print(coeff)

    # # calculate velocity of fluid coming down first incline 
    vel_result = hh.solve_quad(a, b, c)
    vel = vel_result[0]

    # volume flow rate 
    vfr = (vel* math.pi * d["D"]**2) / 4

    # reynolds number 
    Re = (d["rho"]*vel*d["D"]) / d["mu"]

    # friction factor due to fluid viscosity 
    f = 64/ Re

    # head loss 
    h_lo = f * (d["L"] / d["D"]) *  0.5*((vel**2) / d["g"])

    # concerned about head loss as fluid is being pumped up the height h
    h_l = f * (d["h"] / d["D"]) *  0.5*((vel**2) / d["g"])

    # pump work 
    w_pump = vfr * d["rho"] * d["g"] * h_l

    print(f"vfr {vfr},  in gpm: {vfr.magnitude*15850} pump work {w_pump/1000} (in kw) \n")

    # return vel_result
    
    # vel = vel_result[0]

In [27]:
heights = np.array([100, 1000, 10000])
diameters = np.array([0.8, 1.5, 2])

for h in heights:
    for d in diameters:
        d = hh.constants(rho=1030, mu=1, D=d, h=h, beta=1.1 )
        v = calc_properties(d)

pipe height = 100 meter, diam = 0.8 meter, length = 6667.416624483912 meter 
The equation has two solutions: 3.0327485626965256 meter / second or -326.6937497512358 meter / second
vfr 1.5244256967603858 meter ** 3 / second,  in gpm: 24162.147293652113 pump work 23.11599920393854 kilogram * meter ** 2 / second ** 3 (in kw) 

pipe height = 100 meter, diam = 1.5 meter, length = 6667.416624483912 meter 
The equation has two solutions: 9.73294574042967 meter / second or -101.79651941183643 meter / second
vfr 17.19953484520608 meter ** 3 / second,  in gpm: 272612.6272965164 pump work 238.08304271505094 kilogram * meter ** 2 / second ** 3 (in kw) 

pipe height = 100 meter, diam = 2.0 meter, length = 6667.416624483912 meter 
The equation has two solutions: 14.865201798185002 meter / second or -66.65096198835131 meter / second
vfr 46.700408763307784 meter ** 3 / second,  in gpm: 740201.4788984284 pump work 555.3688002594381 kilogram * meter ** 2 / second ** 3 (in kw) 

pipe height = 1000 meter,

In [29]:
# 4 million m^3 / day => s <= minimum flow rate ?
city_fr = (4e6 * hh.m**3 / hh.u.day).to("m**3 / s")
city_fr
# 40 km of movement 
# 2.5 kw - 13.5 kW pumps 

46.29629629629629 <Unit('meter ** 3 / second')>

In [28]:
city_fr*15850 # gpm

733796.2962962962 <Unit('meter ** 3 / second')>

In [339]:

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x.magnitude,
    y=y.magnitude, 
    mode='lines+markers',
))

fig.update_layout(title='Examine Quad - Solving for V',
                   xaxis_title='x ',
                   yaxis_title='y')

# boundaries


In [18]:
# 4 million m^3 / day => s <= minimum flow rate ?
city_fr = (4e6 * hh.m**3 / hh.u.day).to("m**3 / s")
city_fr
# 40 km of movement 
# 2.5 kw - 13.5 kW pumps 

46.29629629629629 <Unit('meter ** 3 / second')>

In [350]:
46

46